In [1]:
### Import libraries
import openai
import random
import os
import time
from datetime import datetime
import pandas as pd
import clingo
from clingo.control import Control
import json
from helper import *
from stories import *
from pipelines import *
import psycopg2

In [2]:
now = datetime.now()
date_time = now.strftime("%Y%m%d")

## OpenAI

In [3]:
set_api_key()

## Connect to Database

In [4]:
conn = get_connection()

## Load Projects

In [5]:
projects = get_useful_projects()

## Functions

In [6]:
def save_in_database(conn, prompt1, prompt2, encoding, story, story_1, story_2, version, file):
    cursor = conn.cursor()
    command = "INSERT INTO ne_linebyline (prompt1, prompt2, encoding, story, story_1, story_2, version, file) VALUES ('%(prompt1)s', '%(prompt2)s', '%(encoding)s', '%(story)s', '%(story_1)s', '%(story_2)s', '%(version)s', '%(file)s');" % {"prompt1": prompt1, "prompt2": prompt2, "encoding": encoding, "story": story, "story_1": story_1, "story_2":story_2, "version":version, "file":file}
    try:
        cursor.execute(command)
        conn.commit()
        cursor.close()
    except Exception as err:
        print(command)
        print(f"Unexpected {err=}, {type(err)=}")
        conn.commit()
        cursor.close()
        pass

In [7]:
def get_2_random_projects(projects,project):
    filtered_projects = [element for element in projects if element != project]
    project_1, project_2 = random.sample(filtered_projects,2)
    return project_1, project_2

In [8]:
def quality_check(name_file, lines_file, previous_messages, instance, deep):
    if deep==0:
        return lines_file[-1]
    with open(name_file,"w") as tmp_file:
        for new_line in lines_file:
            tmp_file.write(new_line)
    models, errors, symbols, mistakes = asp_try_5(name_file,instance)
    #print(f"Errors {errors}. Messages {mistakes}.")
    new_answer = lines_file[-1]
    if len(mistakes) > 0:
        print("Mistake! Last Message : " + str(previous_messages[-1]))
        answer = get_completion_line('''The last rule that you wrote is wrong. The error message is: 
                                    ###" + str(mistakes) + "###.
                                    Write that last rule again, correcting it, without apologizing or writing any explanation.'''
                                    , previous_messages = previous_messages)
        lines_file[-1] = answer
        print("Deep: "+ str(deep) + ". New answer: " +answer)
        new_answer = quality_check(name_file,lines_file,previous_messages, instance, deep-1)
    return new_answer

## Main : From Script, Line4Line, ICL, TryAgain

In [9]:
for project in projects:
    story = project["story"]
    encoding = project["lines"]
    instance = instances_dict[story]
    problem = project["problem"]
    lines = encoding.splitlines()
    new_file = []
    project_1, project_2 = get_2_random_projects(projects,project)
    story_1 = project_1["story"]
    story_2 = project_2["story"]
    content1 = f'''Problem1: {project_1["problem"]}
                    Encoding1: {project_1["lines"]}  '''
    content2 = f'''Problem2: {project_2["problem"]}
                    Encoding2: {project_2["lines"]}  '''
    messages = [{"role":"user","content":content1}]
    messages.append({"role":"user","content":content2})
    messages.append({"role":"user","content":f"Problem1: {problem}"})
    new_file_name = story + "_from_" + story_1 + "_" + story_2 + "_v14_SLIT_" + date_time + ".lp"
    bad_file = False
    for line in lines:
        if line == '\n':
            continue
        if len(line.strip()) > 0 and line.strip()[0] == '%':
            print("---------------------------------------------------")
            new_file.append("\n" + line)
            messages.append({"role":"user","content":line})
            print(line)
            answer = get_completion_line(line,previous_messages=messages)
            print(answer)
            new_file.append("\n" + answer)
            messages.append({"role":"assistant","content":answer})
            if not bad_file:
                new_answer = quality_check(new_file_name,new_file,messages,instance,deep=3)
                new_file[-1] = new_answer
                with open(new_file_name,"w") as tmp_file:
                    for new_line in new_file:
                        tmp_file.write(new_line)
                models, errors, symbols, mistakes = asp_try_5(new_file_name,instance)
                if len(mistakes) > 0:
                    bad_file = True
    os.remove(new_file_name) 
    prompt = '''Given the description of a problem, and a commented line, 
                your task is to translate this commented line in Answer Set Programming. 
                Just write one or a few predicates or rules. Do not try to solve the whole problem at once.'''
    encoding = ' '.join(new_file)
    save_in_database(conn,prompt,'',encoding.replace("'",""),story,story_1, story_2, 14, new_file_name)  

---------------------------------------------------
% Define the plus shape by specifying its four corners: (0,0), (0,1), (1,0), and (1,1).
plus(0,0). plus(0,1). plus(1,0). plus(1,1).
---------------------------------------------------
% Define the four possible directions for movement: up (0,1), down (0,-1), right (1,0), and left (-1,0), for orthogonal connectivity and adjacency checks.
dir(0,1). dir(0,-1). dir(1,0). dir(-1,0).
---------------------------------------------------
% Ensure that exactly N cells adjacent to the hint position (X,Y) are blackened, where the adjacency is defined by the plus shape around the hint position.
:- hint(X,Y,N), N != #count{ DX,DY : plus(DX,DY), black(X+DX,Y+DY) }.
---------------------------------------------------
% Define a path between two white cells (X,Y) and (P,Q) if they are orthogonally adjacent, meaning they share a side and are exactly one unit apart.
path(X,Y,P,Q) :- white(X,Y), white(P,Q), dir(DX,DY), P = X+DX, Q = Y+DY.
---------------

In [10]:
new_file

['\n% Define the non-empty cells (X,Y) in the grid, where X ranges from 1 to the number of rows (R) and Y ranges from 1 to the number of columns (C).',
 '\ncell(X,Y) :- row(X), column(Y), not empty(X,Y).',
 '\n% Define the cells (X,Y) that contain a digit.',
 '\ndigitCell(X,Y,N) :- cell(X,Y), digit(N).',
 '\n% Define the four possible directions for illumination: up (1,0), down (-1,0), right (0,1), and left (0,-1).',
 '\ndirection(1,0).\ndirection(-1,0).\ndirection(0,1).\ndirection(0,-1).',
 '\n% Define "neighbor" of 6 variables for 2 fields if there is a "delta" distance between them TODO',
 'neighbor(X,Y,DX,DY,XX,YY) :- direction(DX,DY), XX=X+DX, YY=Y+DY, cell(X,Y), cell(XX,YY).',
 '\n% Neighbors that are not "digit" are defined as "reach" TODO',
 '\nreach(X,Y,DX,DY,XX,YY) :- neighbor(X,Y,DX,DY,XX,YY), not digitCell(XX,YY,_).',
 '\n% Transitive property of relation "reach": all fields that are neighbours in the same direction, have a relation reach',
 '\nreach(X,Y,DX,DY,XX,YY) :- rea